<a href="https://colab.research.google.com/github/chris69-bit/Kaggle_Competition/blob/main/%F0%9F%92%B2House_Price_Prediction_%F0%9F%8F%98%EF%B8%8F_and_EDA_%F0%9F%94%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

home_data_for_ml_course_path = kagglehub.competition_download('home-data-for-ml-course')

print('Data source import complete.')


# <strong> Import Library </strong>

In [ ]:
%%capture
pip install lazypredict

In [ ]:
import pandas as pd
import numpy as np
import missingno
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler , PowerTransformer , RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split , cross_val_score , KFold

from lazypredict.Supervised import LazyRegressor
import optuna
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import  ExtraTreesRegressor , StackingRegressor , GradientBoostingRegressor

import lightgbm
import xgboost as xgb
import catboost


In [ ]:
# DISPLAY ALL COLUMNS DATASET DATAFRAME
pd.set_option('display.max_columns', None)

# <strong> Load Dataset </strong>

In [ ]:
# LOAD DATAFRAME

train_data = pd.read_table(r'/kaggle/input/home-data-for-ml-course/train.csv', sep=',')
test_data  = pd.read_table(r'/kaggle/input/home-data-for-ml-course/test.csv', sep=',')

print(f'Shape of Train data : {train_data.shape}')
print(f'Shape of Test data  : {test_data.shape}')

In [ ]:
# DISPLAY TRAIN DATA

train_data.head(5)

In [ ]:
test_data.head(5)

## Combine Train and Test Data

In [ ]:
combined_data = pd.concat((train_data, test_data), axis=0)

combined_data

# <strong> Exploratory Data Analysis (EDA) </strong>

## Check Information

In [ ]:
train_data.info()

In [ ]:
test_data.info()

## Check Null Values

In [ ]:
# NULL VALUES COMPARISON

train_null = train_data.isna().sum()
train_null.drop(labels='SalePrice', axis=0, inplace=True)

test_null  = test_data.isna().sum()

train_null.compare(test_null).sort_values(by='self', ascending=False)

There are features with many null values ​​and there are also those with only a few null values.

for numeric features, I will try to fill with 0 value

features with few null values, such as Electrical, MsZoning, Functional, KitchenQual, Exterior2nd, Exterior1st, Utilities, SaleType will be handled with mode values , the rest I will replace with the label 'unknown'.

In [ ]:
# PERCENTAGE OF MISSING VALUES

# CONVERT TO DATAFRAME
missing_value = pd.DataFrame(data=train_data.isna().sum(), index=train_data.columns, columns=['missing'])

# ADD 'PERCENTAGE' COLUMN
missing_value['Percentage'] = (missing_value['missing'] / 1460) * 100

# SORT BY DESCENDING
missing_value = missing_value.sort_values(by='Percentage',ascending=False)

missing_value.head(20)

we can see here, features like PoolQC, Alley, Fence, FireplaceQU, LotFrontage almost all rows are null. we can delete them later

In [ ]:
# VISUALIZE MISSING VALUES

missingno.matrix(df= train_data) , missingno.matrix(df= test_data)

In [ ]:
# VISUALIZE NULL VALUES WITH HEATMAP

missingno.heatmap(df= train_data) , missingno.heatmap(df= test_data)

seen in the heatmap, a value close to 1 indicates that if one feature has a missing value, the other features also tend to have a missing value.

## Check Distribution Data

First, we separate the columns that are categorical and numeric.

In [ ]:
# SEPARATE FEATURE INTO 3 TYPES OF VARIABLE

categorical_feature = train_data.select_dtypes(include='object').columns.tolist()

numerical_feature   = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_feature   = [col for col in numerical_feature if col != 'Id' and col != 'SalePrice']

discrete_feature    = [col for col in numerical_feature if len(train_data[col].unique()) < 25]
continuous_feature  = [col for col in numerical_feature if col not in discrete_feature]

print(f'Number of Categorical Feature : {len(categorical_feature)}')
print(f'Number of Numerical Feature   : {len(numerical_feature)}')
print(f'Number of Discrete Feature    : {len(discrete_feature)}')
print(f'Number of Continous Feature   : {len(continuous_feature)}')

### Comparison Categorical Feature

In [ ]:
# VISUALIZE DISTRIBUTION OF CATEGORICAL FEATURE

fig , axes = plt.subplots(nrows= 9, ncols=5, figsize=(30,30))

for i, feature in enumerate(categorical_feature):
    sns.histplot(data= train_data, x=feature, ax=axes[i%9, i//9], color='purple')   # DISTRIBUTION FOR TRAIN DATA
    sns.histplot(data= test_data, x=feature, ax=axes[i%9, i//9], color='lightgreen')  # DISTRIBUTION FOR TEST DATA

The data distribution between training data and test data is almost the same, except for PoolQC

Some Feature have Dominants Label ,So we drop features that are only dominated by 1 label, such as RoofMatl, Street, GarageCond, Condition2, Utilities, Heating.

### Comparison Discrete Feature

In [ ]:
# VISUALIZE DISTRIBUTION OF DISCRETE FEATURE

fig , axes = plt.subplots(nrows=3, ncols=6, figsize=(20,10))

for i , feature in enumerate(discrete_feature):
    sns.histplot(data= train_data, x= feature, ax=axes[i%3, i//3], color='blue')
    sns.histplot(data= test_data,  x= feature, ax=axes[i%3, i//3], color='orange')

plt.show()

some features are dominated by 0 values, such as LowQualFinSF, MiscVal, 3SsnPorch, PoolArea. I will drop those columns

### Comparison Continuous Feature

In [ ]:
# VISUALIZATION DISTRIBUTION OF CONTINUOUS FEATURE

fig , axes = plt.subplots(nrows=4, ncols=5, figsize=(20,15))

for i, feature in enumerate(continuous_feature):
    sns.histplot(data= train_data, x=feature, ax=axes[i%4, i//4], color='darkblue')
    sns.histplot(data= test_data,  x=feature, ax=axes[i%4,i//4],  color='gold')

plt.show()

some features have a negative skewed distribution

let's calculate how skew it is

In [ ]:
# CALCULATE HOW SKEW IT IS

skewness_train = train_data[numerical_feature].skew().sort_values(ascending=False)
skewness_test  = test_data[numerical_feature].skew().sort_values(ascending=False)

avg_skewness = (skewness_train + skewness_test) / 2
avg_skewness = avg_skewness.sort_values(ascending=False)

print(avg_skewness)

any skew value is more than 1 or -1 we will do Transformation

ok now lets check SalePrice distribution

In [ ]:
# SALEPRICE DISTRIBUTION

sns.histplot(data=train_data, x='SalePrice')

The distribution of 'SalePrice' shows negative skewness, so we will do a log transformation later.

## Correlation Comparison

In [ ]:
# CHECK CORRELATION EVERY FEATURE

# SET NUMERIC FEATURE
numeric_data = pd.DataFrame()

for feature in numerical_feature:
    numeric_data[feature] = train_data[feature]

corr_data = numeric_data.corr(method='pearson')

plt.figure(figsize=(30,30))
sns.heatmap(data= corr_data, cmap='coolwarm', annot=True, fmt='.2g')

conclusion of correlation:

1. There is a strong correlation between:
    - GarageArea and GarageCars.
    - 1stFlrSF and TotalBsmtSF.
    - GrLivArea and TotRmsAbvGrd.

we can consider removing one of them or using PCA to select the most important features

2. create new features such as:
    - df['GarageEfficiency'] = df['GarageArea'] / df['GarageCars']
    - df['LivabilityScore'] = df['GrLivArea'] + (df['FullBath'] * 2) + (df['HalfBath'])

OK, next we check the correlation with the target features.

In [ ]:
# CORRELATION WITH SALEPRICE

numeric_data['SalePrice'] = train_data['SalePrice']  # ADD SALEPRICE COLUMN

corr_data = numeric_data.corr(method='pearson')
corr_data = corr_data[['SalePrice']]          # ONLY SHOWS CORRELATION FOR SALEPRICE FEATURE


plt.figure(figsize=(7,10))
sns.heatmap(data=corr_data, cmap='coolwarm', annot=True, fmt='.2g')

1. Highly correlated variables with SalePrice(positive, >0.5):

    - OverallQual (0.79) <br>
    - GrLivArea (0.71)  <br>
    - GarageCars (0.64) <br>
    - 1stFlrSF (0.61)   <br>
    - TotalBsmtSF (0.61) <br>
These variables are strong candidates for inclusion in the model because they provide significant information to predict home price. <br> <br>

2. Moderate correlations with SalePrice(0.3–0.5):

    - YearBuilt (0.52) <br>
    - YearRemodAdd (0.51) <br>
    - FullBath (0.56)  <br>
    - Fireplaces (0.47) <br>
    - TotRmsAbvGrd (0.53) <br>
    - MasVnrArea (0.48) <br>

These variables provide additional information that can help the model improve prediction accuracy, but are not as strong as the highly correlated variables. <br> <br>

3. create new feature TotalArea = GrLivArea + TotalBsmtSF: Adds the area of ​​living space and basement.

ok now check the linearity of the feature against saleprice

In [ ]:
# LINEARITY USING SCATTER PLOT

fig , axes = plt.subplots(nrows=7, ncols=6, figsize=(37,25))

for i , feature in enumerate(numerical_feature):
    sns.regplot(data= train_data, x= feature, y= 'SalePrice', ax= axes[i%7, i//7])

plt.show()


From the data above, there are non-linear relationships such as: <br>
GrLivArea and SalePrice,  <br>
BsmtFinSF1 and SalePrice, <br>
TotalBsmtSF and SalePrice, <br>
LotFrontage and SalePrice, <br>
LotArea and SalePrice,     <br>
GarageArea and SalePrice   <br>


ok next check is there any relation between Yrsold and SalePrice

In [ ]:
combined_data.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('Year Sold')
plt.ylabel('House Price')
plt.title('House price vs YearSold')

There is a negative correlation between YrSold and SalePrice. This means that YrSold has an effect on SalePrice.

# <strong> Feature Engineering </strong>

## Drop Columns

In [ ]:

# FIRST DROP COLUMNS WITH MANY NULL VALUES
cols_with_many_null = ['PoolQC', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage']
combined_data.drop(labels=cols_with_many_null, axis=1, inplace=True)

# NEXT DROP COLUMNS WITH MANY ZERO VALUES
cols_with_many_zero = ['LowQualFinSF', 'MiscVal', '3SsnPorch', 'PoolArea']
combined_data.drop(labels=cols_with_many_zero, axis=1, inplace=True)

# LAST DROP COLUMNS WITH DOMINANT 1 LABEL
cols_with_dominant_label = ['Id','RoofMatl', 'Street', 'Condition2', 'Utilities', 'Heating']
combined_data.drop(labels=cols_with_dominant_label, axis=1, inplace=True)

combined_data.columns  , len(combined_data.columns)   # CHECK FINAL COLUMNS AFTER DROPPED

## Temporal Variable

Calculating the age of the building, renovations, and year of construction of the garage based on the year of sale

In [ ]:
# FIND ALL DATE FEATURE
year_feature = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

for feature in year_feature:
    combined_data[feature] = combined_data['YrSold'] - combined_data[feature]

combined_data[year_feature].head(5)

## Fill Missing Values

In [ ]:
# FILL NUMERICAL MISSING VALUES WITH ZERO VALUES
numerical_feature = combined_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
for feature in numerical_feature:
    combined_data[feature] = combined_data[feature].fillna(0)

# FILL CONTINUOUS MISSING VALUES
dropped_cols = cols_with_dominant_label + cols_with_many_null
mode_feature = ['Electrical', 'MsZoning', 'Functional', 'KitchenQual', 'Exterior2nd', 'Exterior1st', 'Utilities', 'SaleType']


for feature in categorical_feature:
    if feature not in dropped_cols:
        if feature not in mode_feature:
            combined_data[feature] = combined_data[feature].fillna('Unknown')
        else:
            combined_data[feature] = combined_data[feature].fillna(combined_data[feature].mode()[0])

In [ ]:
# CHECK IT BACK

combined_data.info()

## Combining Variables and Create new Feature

In [ ]:

# ADD NEW FEATURE 'GarageEfficiency'
combined_data['GarageEfficiency'] = combined_data['GarageArea'] / (combined_data['GarageCars'] + 1)   # +1 TO AVOID DIVISION BY ZERO

# ADD NEW FEATURE
#combined_data['LivabilityScore'] = combined_data['GrLivArea'] + (combined_data['FullBath'] * 2) + (combined_data['HalfBath'])

# ADD NEW FEATURE TotalArea
combined_data['TotalArea'] = combined_data['GrLivArea'] + combined_data['TotalBsmtSF']

In [ ]:
# CHECK NEW FEATURE

combined_data[['GarageEfficiency', 'TotalArea']].head(5)

In [ ]:
# CHECK CORRELATION OF NEW FEATURE

new_feature = combined_data[['SalePrice','GarageEfficiency','GarageArea','GarageCars','TotalArea','GrLivArea','TotalBsmtSF']]

corr_new_feature = new_feature.corr(method='spearman')

sns.heatmap(data=corr_new_feature, cmap='coolwarm', annot=True, fmt='.2g')

## Overcome Multicolinearity

In [ ]:
# USING PCA TO ADDRESSING MULTICOLINEARITY


zscore = StandardScaler()

# NORMALIZATION
multi_cols_1 = combined_data[['GarageArea','GarageCars']]   # COLUMN THAT HAS MULTICOLINEARITY
multi_cols_1 = zscore.fit_transform(multi_cols_1)

# FIND PRINCIPAL COMPONENT OPTIMAL
pca_1  = PCA(n_components=None)
pca_1.fit_transform(multi_cols_1)


# NORMALIZATION
multi_cols_2 = combined_data[['1stFlrSF', 'TotalBsmtSF']]
multi_cols_2 = zscore.fit_transform(multi_cols_2)

# FIND PRINCIPAL COMPONENT OPTIMAL
pca_2  = PCA(n_components=None)
pca_2.fit_transform(multi_cols_2)


# NORMALIZATION
multi_cols_3 = combined_data[['GrLivArea', 'TotRmsAbvGrd']]
multi_cols_3 = zscore.fit_transform(multi_cols_3)

# FIND PRINCIPAL COMPONENT OPTIMAL
pca_3  = PCA(n_components=None)
pca_3.fit_transform(multi_cols_3)

for pca in [pca_1, pca_2, pca_3]:
    print(f'Number of Components : {pca.n_components_}')
    print(f'Ratio every Component / PC : {pca.explained_variance_ratio_}\n')
    print(f'PCA Components : \n{pca.components_}\n')
    print('---------------------------------')

PC1 explains up to 90%, so we only take PC1

In [ ]:
# CHOOSE PC1

pca_1  = PCA(n_components=1)
multi_cols_1 = pca_1.fit_transform(multi_cols_1)

pca_2  = PCA(n_components=1)
multi_cols_2 = pca_2.fit_transform(multi_cols_2)

pca_3  = PCA(n_components=1)
multi_cols_3 = pca_3.fit_transform(multi_cols_3)

# ADD PC1 TO DATAFRAME
combined_data['multi_cols_1'] = multi_cols_1.ravel()
combined_data['multi_cols_2'] = multi_cols_2.ravel()
combined_data['multi_cols_3'] = multi_cols_3.ravel()


In [ ]:
combined_data.head(5)

In [ ]:
# CHECK BACK HEATMAP

multi = combined_data[['SalePrice', 'multi_cols_1','GarageCars','GarageArea', 'multi_cols_2','1stFlrSF', 'TotalBsmtSF', 'multi_cols_3', 'GrLivArea', 'TotRmsAbvGrd']]

multiq = multi.corr(method='spearman')

sns.heatmap(data=multiq, annot=True, fmt='.2g')

we have successfully overcome multicollinearity, next we drop the multicollinearity feature

In [ ]:
# DROP MULTICOLINEARITY FEATURE

multi_corr = ['GarageCars','GarageArea','1stFlrSF', 'TotalBsmtSF','GrLivArea', 'TotRmsAbvGrd']

combined_data.drop(labels= multi_corr, axis=1, inplace=True )

combined_data.head(5)

## Feature Transformation

In [ ]:
# APPLY POWER TRANSFORMATION TO FEATURE THAT SKEWNESS VALUE > 1 OR < -1

# SELECT ALL NUMERICAL DATA
transform_data      = combined_data.select_dtypes(include=['int64', 'float64'])
continuous_feature  = [col for col in transform_data if len(transform_data[col].unique()) > 25]    # SELECT ONLY CONTINUOUS FEATURE
transform_data = combined_data[continuous_feature]
transform_data.drop(labels=['SalePrice'], axis=1 , inplace=True)

# CHECK SKEWNESS
skewness = transform_data.skew().sort_values(ascending=False)

# APPLY TRANSFORM FOR FEATURE HAVE SKEWNESS > 1 OR < -1
# cols_to_transform = skewness[(skewness > 1) | (skewness < -1)].index
cols_to_transform = ['LotArea', 'TotalArea', 'TotalArea', 'multi_cols_2', 'BsmtUnfSF']  # CHOSEN FEATURE TO TRANSFORM

for col in cols_to_transform:
    yeo_johnson = PowerTransformer(method='yeo-johnson', standardize=True, copy=True)
    combined_data[[col]] = yeo_johnson.fit_transform(combined_data[[col]])


In [ ]:
# VISUALIZE AFTER TRANSFORMATION
fig , axes = plt.subplots(nrows=1, ncols=5, figsize=(14,4))

for i , feature in enumerate(cols_to_transform):
    sns.histplot(data=combined_data, x=feature, ax=axes[i%5])

plt.show()

## One Hot Encoder

In [ ]:
combined_data = pd.get_dummies(combined_data).reset_index(drop=True)

combined_data

### Re-Split the Dataset

In [ ]:
new_train_data = combined_data.iloc[:len(train_data), :]
new_test_data  = combined_data.iloc[len(train_data):, :]

x_train = new_train_data.drop(labels=['SalePrice'], axis=1)

# LOG TRANSFORMATION FOR TARGET FEATURE (SalePrice)
y_train = np.log1p(new_train_data['SalePrice'])

x_test = new_test_data.drop(labels=['SalePrice'], axis=1)


x_train.shape , y_train.shape, x_test.shape

## Feature Scaling

before we do normalization , let's take a look at the data distribution for the last time

In [ ]:

# SELECT ALL NUMERICAL FEATURE
numerical_feature = ['MSSubClass','LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','2ndFlrSF','BsmtFullBath',
                     'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','Fireplaces','GarageYrBlt','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch',
                     'MoSold','YrSold','GarageEfficiency','TotalArea','multi_cols_1','multi_cols_2','multi_cols_3']


# VISUALIZE IT
fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(25,20))
for i , feature in enumerate(numerical_feature):
    sns.histplot(data= x_train , x=feature , ax=axes[i%5, i//5])

# BOXPLOT
fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(25,20))
for i,feature in enumerate(numerical_feature):
    sns.boxplot(data=x_train, x=feature, ax=axes[i%5, i//5])

plt.show()

We will perform robust scaling normalization for features that have many outliers and are not normally distributed. and use zscore normalization for features that are almost normally distributed.

In [ ]:
# NORMALIZATION

# CHOOSE COLUMNS TO NORMALIZE
cols_to_robust = ['MSSubClass','YearRemodAdd', '2ndFlrSF','BedroomAbvGr','OpenPorchSF','MasVnrArea','EnclosedPorch','BsmtFinSF1','ScreenPorch','BsmtFinSF2','GarageYrBlt','YearBuilt','WoodDeckSF']
cols_to_zscore = ['GarageEfficiency','LotArea','TotalArea','OverallQual','multi_cols_1','OverallCond','multi_cols_2','BsmtUnfSF','multi_cols_3']

# ROBUST SCALING NORM
robust = RobustScaler()
robust.fit(x_train[cols_to_robust])

x_train[cols_to_robust] = robust.transform(x_train[cols_to_robust])
x_test[cols_to_robust]  = robust.transform(x_test[cols_to_robust])

# ZSCORE NORM
zscore = StandardScaler()
zscore.fit(x_train[cols_to_zscore])

x_train[cols_to_zscore] = zscore.transform(x_train[cols_to_zscore])
x_test[cols_to_zscore]  = zscore.transform(x_test[cols_to_zscore])

x_train.shape , x_test.shape

# <strong> Model Development </strong>

## Lazy Predict

In [ ]:
# BUILDING LAZY PREDICT TO FIND BEST MODEL

x_train_lazy , x_test_lazy , y_train_lazy , y_test_lazy = train_test_split(x_train, y_train, test_size=0.2, random_state=12, shuffle=True)


lazy_model = LazyRegressor(verbose=0, random_state=12, regressors='all')
train_lazy , test_lazy = lazy_model.fit(x_train_lazy, x_test_lazy, y_train_lazy, y_test_lazy)
test_lazy


we can see above the best algorithm. Ridge and Linear Regression are better compare to each other. we can use them for Stacking-Models. <br>

so, i want use <u>_Linear Model_</u> + <u>_Non-Linear Model_</u> Combination for Stacking Model . <br>

Here my Combination :
- <strong>Ridge Regression</strong> (Linear Model)
- <strong>Linear Regression</strong>  (Linear Model)
- <strong>Lasso Regression </strong>   (Linear Model)
- <strong>Gradient Boosting Machine </strong>   (Non-Linear Model)
- <strong>XGBoost </strong>  (Non-Linear Model)
- <strong>LightGBM </strong> (Non-Linear Model)
- <strong>CatBoost </strong> (Non-Linear Model)


## Find Best Hyperparameter using Optuna

## Ridge Regression

Find best Alpha and Max_iter for Ridge using Optuna

In [ ]:
def Ridge_objective(trial):

    # SET HYPERPARAMETERS VALUE RANGE
    alpha    = trial.suggest_float('alpha',0.1, 25)
    max_iter = trial.suggest_int('max_iter',150, 2000)

    # SET MODEL
    model = Ridge(alpha=alpha,
                  max_iter=max_iter,
                  random_state=12)

    # SCORE METRICS
    score = cross_val_score(estimator= model , X = x_train, y= y_train, scoring='neg_root_mean_squared_error')

    return score.mean()

# BUILD A OPTUNA
#study = optuna.create_study(direction='maximize')
#study.optimize(func= Ridge_objective, n_trials=100)

#print(f'Best CV    : {study.best_params}')
#print(f'Best Score : {study.best_value}')

# GET BEST HYPERPARAMETERS
#ridge_best_params = study.best_params

'''
Best CV    : {'alpha': 10.92149084623178, 'max_iter': 1112}
Best Score : -0.12549632247936676
'''

'''
Best CV    : {'alpha': 10.907628470160567, 'max_iter': 685}
Best Score : -0.1254963234836375
'''

fit best hyperparameter and create Ridge model

In [ ]:
# BUILD A MODEL

ridge_best_params = {'alpha': 10.907628470160567, 'max_iter': 685}

ridge = Ridge(fit_intercept=True, solver='auto', random_state=12, **ridge_best_params)
ridge.fit(x_train, y_train)


### Feature Importance for Ridge

In [ ]:
# THIS VARIABLE IS TO SEE WHICH FEATURES ARE THE MOST USEFUL AND THE MOST USELESS

useless_feature = {}   # CAPTURE coefficient VALUE CLOSE TO ZERO
useful_feature  = {}   # CAPTURE HIGH coefficient VALUE

# THIS VARIABLE WILL BE VISUALIZE AT THE END

In [ ]:
# CHECK FEATURE IMPORTANCES

coefficients = ridge.coef_

features_importance = pd.DataFrame({
    'feature': ridge.feature_names_in_,
    'coefficient': coefficients
})

features_importance = features_importance.sort_values(by='coefficient', ascending=False)

plt.figure(figsize=(10,60))
plt.barh(y=features_importance['feature'], width=features_importance['coefficient'])



lets display coefficient that has value close to 0

In [ ]:

features_importance['coefficient'] = abs(features_importance['coefficient'])   # FIRST,CONVERT COEF TO POSITIVE VALUE

zero = features_importance[features_importance['coefficient'] < 0.001 ].sort_values(by='feature', ascending=True)


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 0.02


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero


confuse? ok lets print 'useful_feature' variable

In [ ]:
useful_feature

useful_feature.keys() contain name of feature/columns, and useful_feature.values() contains coefficient/weight of features

## Lasso Regression

In [ ]:
def lasso_objective(trial):

    # SET HYPERPARAMETERS VALUE RANGE
    alpha    = trial.suggest_float('alpha',0.1, 25)
    max_iter = trial.suggest_int('max_iter',150, 2000)

    # SET MODEL
    model = Lasso(alpha=alpha,
                  max_iter=max_iter,
                  random_state=10)

    # SCORE METRICS
    score = cross_val_score(estimator= model , X = x_train, y= y_train, scoring='neg_root_mean_squared_error')

    return score.mean()


#study = optuna.create_study(direction='maximize')
#study.optimize(func= lasso_objective, n_trials=100)

#print(f'Best CV    : {study.best_params}')
#print(f'Best Score : {study.best_value}')

#lasso_best_params = study.best_params

'''
Best CV    : {'alpha': 0.10570252247670833, 'max_iter': 1386}
Best Score : -0.21352962715534524 at trial 61
'''

'''
Best CV    : {'alpha': 0.10064802295354133, 'max_iter': 1839}
Best Score : -0.21014074025313523
'''

In [ ]:
lasso_best_params = {'alpha': 0.10064802295354133, 'max_iter': 1839}

# BUILD A MODEL
lasso = Lasso(fit_intercept=True, random_state=12, **lasso_best_params)
lasso.fit(x_train, y_train)

### Feature Importances Lasso Regression

In [ ]:
# CHECK FEATURE IMPORTANCES

coefficients = lasso.coef_  # GET WEIGHT / COEFFICIENT

features_importance = pd.DataFrame({
    'feature': lasso.feature_names_in_,
    'coefficient': coefficients
})

features_importance = features_importance.sort_values(by='coefficient', ascending=False)

plt.figure(figsize=(10,60))
plt.barh(y=features_importance['feature'], width=features_importance['coefficient'])


almost all of its features have zero coefficient

In [ ]:

features_importance['coefficient'] = abs(features_importance['coefficient'])   # FIRST,CONVERT COEF TO POSITIVE VALUE

zero = features_importance[features_importance['coefficient'] == 0 ].sort_values(by='feature', ascending=True)


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 0.02


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero

## Linear Regression

In [ ]:
linear = LinearRegression(fit_intercept=True, positive=False)
linear.fit(x_train, y_train)

### Feature Importance for Linear Regression

In [ ]:

coefficients = linear.coef_

features_importance = pd.DataFrame({
    'feature' : linear.feature_names_in_,
    'coefficient' : coefficients
})
features_importance = features_importance.sort_values(by='coefficient', ascending=False)  # SORT IT TO DESCENDING

plt.figure(figsize=(10,50))

plt.barh(y=features_importance['feature'], width=features_importance['coefficient'])
plt.show()

show feature with values close to 0

In [ ]:
features_importance['coefficient'] = abs(features_importance['coefficient'])   # FIRST,CONVERT COEF TO POSITIVE VALUE

zero = features_importance[features_importance['coefficient'] < 0.001 ].sort_values(by='feature', ascending=True)


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 0.02


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero

## Gradient Boosting Machine

In [ ]:
# GRADIENT BOOSTING MACHINE

def gbr_objective(trial):

    # SET HYPERPARAMETER VALUE RANGE
    n_estimators     = trial.suggest_int('n_estimators', 50,2000)
    learning_rate    = trial.suggest_uniform('learning_rate', 0.001, 1)
    max_depth        = trial.suggest_int('max_depth', 2, 16)
    min_sample_split = trial.suggest_int('min_samples_split', 2, 25)
    max_features     = trial.suggest_int('max_features', 5, 60)

    # DECLARATE THE MODEL AND ITS HYPERPARAMETERS
    gbr = GradientBoostingRegressor(
        n_estimators      = n_estimators,
        learning_rate     = learning_rate,
        max_depth         = max_depth,
        min_samples_split = min_sample_split,
        max_features      = max_features,
        random_state      = 12
        )

    # CREATE SCORE EVALUATION
    score = cross_val_score(estimator= gbr, X= x_train, y= y_train, scoring='neg_root_mean_squared_error')

    return score.mean()

# BUILD OPTUNA
#study = optuna.create_study(direction='maximize')

# START TO FIND BEST HYPERPARAMETERS
#study.optimize(func= gbr_objective, n_trials=200)

#print(f'Best Hyperparameter : {study.best_params}')
#print(f'Best Score          : {study.best_value}')

# TAKE THE BEST PARAMETERS
#gbr_best_params = study.best_params

'''
Best Hyperparameter : {'n_estimators': 1899, 'learning_rate': 0.054138183966776846, 'max_depth': 3, 'min_samples_split': 14, 'max_features': 26}
Best Score          : -0.11692210503959213 at trial 97
'''

''' BEST AT KAGGLE
Best Params : {'n_estimators': 1552, 'learning_rate': 0.03236403825303992, 'max_depth': 3, 'min_samples_split': 14, 'max_features': 46}
Best Score  : -0.11735617999331582 at trial 72
'''


In [ ]:
# FIT MODEL GBM

gbr_best_params = {'n_estimators': 1552, 'learning_rate': 0.03236403825303992, 'max_depth': 3, 'min_samples_split': 14, 'max_features': 46}
gbm = GradientBoostingRegressor(random_state=12, **gbr_best_params)
gbm.fit(x_train,y_train)

### Feature Importance for GBM Model

In [ ]:

plt.figure(figsize=(12,50))

features_importance = pd.DataFrame({
    'feature' : gbm.feature_names_in_,
    'coefficient' : gbm.feature_importances_
})

features_importance = features_importance.sort_values(by='coefficient', ascending= True)
plt.barh(y= features_importance['feature'], width= features_importance['coefficient'], color='skyblue')

There are too many coefficients that are close to zero, let's display the coefficients that have a value of 0.

In [ ]:
features_importance[features_importance['coefficient'] < 0.00001 ].sort_values(by='feature', ascending=True)

In [ ]:

zero = features_importance[features_importance['coefficient'] < 0.00001 ].sort_values(by='feature', ascending=True) # SELECT ALL FEATURE THAT WEIGHT < 0.001


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 0.002


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero

## XGBoost

In [ ]:
def xg_objective(trial):

    n_estimators    = trial.suggest_int('n_estimators', 100, 2000)
    learning_rate   = trial.suggest_uniform('learning_rate', 0.01, 1)
    max_depth       = trial.suggest_int('max_depth', 2, 20)
    min_child_weight= trial.suggest_float('min_child_weight', 0.4, 10)
    subsample       = trial.suggest_uniform('subsample', 0.35, 1)
    gamma           = trial.suggest_float('gamma', 0.1, 1)
    reg_alpha       = trial.suggest_float('reg_alpha', 0.01, 15)
    reg_lambda      = trial.suggest_float('reg_lambda', 0.01, 15)

    xgboost = xgb.XGBRegressor(
        n_estimators     = n_estimators,
        learning_rate    = learning_rate,
        max_depth        = max_depth,
        min_child_weight = min_child_weight,
        subsample        = subsample,
        gamma            = gamma,
        reg_alpha        = reg_alpha,
        reg_lambda       = reg_lambda,
        random_state     = 12
    )

    score = cross_val_score(estimator= xgboost, X = x_train, y = y_train, scoring='neg_root_mean_squared_log_error')

    return score.mean()


#study = optuna.create_study(direction='maximize')
#study.optimize(func= xg_objective, n_trials= 200)

#xgb_best_params = study.best_params

'''
Best Params : {'n_estimators': 1738, 'learning_rate': 0.10819163170249219, 'max_depth': 19, 'min_child_weight': 1.0918881759107903, 'subsample': 0.46270891531334185,
                'gamma': 0.10028845351448364, 'reg_alpha': 0.3534965264352544, 'reg_lambda': 12.97887001192773}
Best Score  : -0.009996840517023025 at trial 137
'''

In [ ]:

xgb_best_params = {'n_estimators': 1552, 'learning_rate': 0.03236403825303992, 'max_depth': 3, 'min_samples_split': 14, 'max_features': 46}

xgboost = xgb.XGBRegressor(random_state=12, **xgb_best_params)
xgboost.fit(x_train, y_train)

### Feature Importance for XGBoost

In [ ]:
# SHOW FEATURE IMPORTANCES

plt.figure(figsize=(12,50))

features_importance = pd.DataFrame({
    'feature' : xgboost.feature_names_in_,
    'coefficient' : xgboost.feature_importances_
})

features_importance = features_importance.sort_values(by='coefficient', ascending= True)
plt.barh(y= features_importance['feature'], width= features_importance['coefficient'], color='skyblue')

same as GBM , lets show coefficient that has value 0

In [ ]:
zero = features_importance[features_importance['coefficient'] == 0 ].sort_values(by='feature', ascending=True) # ALL FEATURE THAT WEIGHT < 0.001


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 0.01


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero

## CatBoost

In [ ]:
def catboost_objective(trial):

    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'max_bin': trial.suggest_int('max_bin', 10, 500),
        'eval_metric': 'RMSE',

    }

    cat = catboost.CatBoostRegressor(**params, random_state=12, verbose=0)

    score = cross_val_score(estimator=cat, X= x_train, y= y_train, scoring='neg_root_mean_squared_error')

    return score.mean()

#study = optuna.create_study(direction='maximize')
#study.optimize(func=catboost_objective, n_trials=50)

#print(f'Best Params : {study.best_params}')
#print(f'Best Scores : {study.best_value}')


#catboost_best_params = study.best_params

'''
Best Params : {'iterations': 940, 'learning_rate': 0.05397757917685036, 'depth': 4, 'l2_leaf_reg': 9.88267503317357e-05, 'subsample': 0.9616596331860532,
               'colsample_bylevel': 0.6942581356179441, 'min_data_in_leaf': 61, 'max_bin': 197}
Best Scores : -0.11600213030799092
'''

In [ ]:
%%capture
catboost_best_params = {'iterations': 940, 'learning_rate': 0.05397757917685036, 'depth': 4, 'l2_leaf_reg': 9.88267503317357e-05, 'subsample': 0.9616596331860532,
               'colsample_bylevel': 0.6942581356179441, 'min_data_in_leaf': 61, 'max_bin': 197}

cat = catboost.CatBoostRegressor(random_state=12, devices=0,**catboost_best_params)
cat.fit(x_train, y_train)

### Show Feature Importance for CatBoost

In [ ]:
# SHOW FEATURE IMPORTANCES

plt.figure(figsize=(12,50))

features_importance = pd.DataFrame({
    'feature' : x_train.columns,
    'coefficient' : cat.feature_importances_
})

features_importance = features_importance.sort_values(by='coefficient', ascending= True)
plt.barh(y= features_importance['feature'], width= features_importance['coefficient'], color='skyblue')

Display Feature that has coefficient 0

In [ ]:
zero = features_importance[features_importance['coefficient'] == 0 ].sort_values(by='feature', ascending=True) # ALL FEATURE THAT WEIGHT < 0.001


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 0.3


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero

## LightGBM

In [ ]:


# FUNCTION TO FIND BEST HYPERPARAMETER
def lgbm_objective(trial):

    # SET HYPERPARAMETERS VALUE RANGE
    num_leaves        = trial.suggest_int('num_leaves',20,100)
    max_depth         = trial.suggest_int('max_depth', 2, 15)
    learning_rate     = trial.suggest_float('learning_rate', 0.001, 0.8)
    n_estimators      = trial.suggest_int('n_estimators', 100, 2000)
    min_child_weight  = trial.suggest_int('min_child_weight', 0.5, 5)
    min_child_samples = trial.suggest_int('min_child_samples', 7, 20)
    subsample         = trial.suggest_uniform('subsample', 0.4, 1)
    reg_alpha         = trial.suggest_float('reg_alpha', 0.02, 15)
    reg_lambda        = trial.suggest_float('reg_lambda', 0.02, 15)

    # DECLARATE LGBM MODEL
    lgbm = lightgbm.LGBMRegressor(boosting_type    ='gbdt',
                             num_leaves       = num_leaves,
                             max_depth        = max_depth,
                             learning_rate    = learning_rate,
                             n_estimators     = n_estimators,
                             min_child_weight = min_child_weight,
                             min_child_samples= min_child_samples,
                             subsample        = subsample,
                             reg_alpha        = reg_alpha,
                             reg_lambda       = reg_lambda)

    # METRICS EVALUATION
    score = cross_val_score(estimator= lgbm, X= x_train, y= y_train, scoring='neg_root_mean_squared_error')

    return score.mean()

# BUILD AND FIT OPTUNA
#study = optuna.create_study(direction='maximize')
#study.optimize(func= lgbm_objective, n_trials=200)

#print(f'Best Params : {study.best_params}')
#print(f'Best Scores : {study.best_value}')

# BEST PARAMS
#lgbm_best_params = study.best_params

'''
Best Params : {'num_leaves': 92, 'max_depth': 2, 'learning_rate': 0.059514026241983445, 'n_estimators': 1524, 'min_child_weight': 3, 'min_child_samples': 17, 'subsample': 0.48984666486323647, 'reg_alpha': 0.033788563133902494, 'reg_lambda': 4.040907598613931}
Best Scores : -0.1249959965787046
'''

In [ ]:
%%capture
# BUILD AND FIT MODEL WITH BEST HYPERPARAMETERS

lgbm_best_params = {'num_leaves': 92, 'max_depth': 2, 'learning_rate': 0.059514026241983445, 'n_estimators': 1524, 'min_child_weight': 3, 'min_child_samples': 17,
                    'subsample': 0.48984666486323647, 'reg_alpha': 0.033788563133902494, 'reg_lambda': 4.040907598613931}

lgbm = lightgbm.LGBMRegressor(random_state=12, **lgbm_best_params)
lgbm.fit(x_train, y_train)

### Feature Importance for LGBM

In [ ]:
# SHOW FEATURE IMPORTANCES

plt.figure(figsize=(12,50))

features_importance = pd.DataFrame({
    'feature' : lgbm.feature_names_in_,
    'coefficient' : lgbm.feature_importances_
})

features_importance = features_importance.sort_values(by='coefficient', ascending= True)
plt.barh(y= features_importance['feature'], width= features_importance['coefficient'], color='skyblue')

display coefficient that very close to 0

In [ ]:
zero = features_importance[features_importance['coefficient'] == 0 ].sort_values(by='feature', ascending=True) # ALL FEATURE THAT WEIGHT < 0.001


# PUT CLOSE TO ZERO COEF INTO DICT
for feature in zero['feature']:
    useless_feature[feature] = useless_feature.get(feature,0) + 1



# PUT THE HIGH COEFFICIENT INTO DICT
threshold = 1


high_coef = features_importance[features_importance['coefficient'] >= threshold].sort_values(by='coefficient', ascending=False) # SORT IT DESCENDING
high_coef = high_coef.reset_index(drop=True)

for i, feature in enumerate(high_coef['feature']):
    # GIVE VALUE/POINT ACCORDING TO THEIR WEIGHT
    useful_feature[feature] = useful_feature.get(feature, 0) + high_coef['coefficient'][i] # COEFFICIENTS WITH HIGHER WEIGHT WILL GET MORE POINTS/VALUES

# DISPLAY FEATURE THAT HAS VALUE CLOSE TO ZERO
zero

## Create Stacking Regressor

In [ ]:
%%capture
cv_fold = KFold(n_splits= 10, shuffle=True, random_state=12)

# CREATE STACKING REGRESSOR
model = StackingRegressor(
    estimators=[
        ('Ridge', ridge),
        ('Lasso', lasso),
        #('LinearRegression', linear),       # USING LINEAR REGRESSION CAUSE A MODEL GET WORSE
        ('GradientBoostingRegressor', gbm),
        ('xgb', xgboost),
        ('lightgbm', lgbm),
        ('catboost', cat)
    ],
        cv=cv_fold
)

model.fit(x_train, y_train)

OK, now let's visualize which model has the bigger contribution for Meta-Model

In [ ]:
import matplotlib.pyplot as plt

# BASE MODEL WE TRAINED
base_models = [ 'Ridge', 'Lasso', 'GBM', 'XGBOOST', 'LGBM', 'CatBoost']

meta_model = model.final_estimator_
coefficients = meta_model.coef_

plt.figure(figsize=(10,6))
plt.bar(base_models, coefficients)
plt.xlabel('Base Models')
plt.ylabel('Koefisien')
plt.title('Most Important Base-Model')
plt.show()

last thing , we visualize the most useful feature and the most uselesss feature for our model

In [ ]:

top_50_useless = dict(sorted(useless_feature.items(), key=lambda item: item[1], reverse=True)[:50])
top_20_useful  = dict(sorted(useful_feature.items(), key=lambda item: item[1], reverse=True)[:20])


plt.figure(figsize=(20,5))
sns.barplot(x= list(top_50_useless.keys()), y= list(top_50_useless.values()), palette='inferno')
plt.title('Top 50 Useless Feature')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(12,5))
sns.barplot(x= list(top_20_useful.keys()), y= list(top_20_useful.values()), palette='plasma')
plt.title('Top 20 Useful Feature')
plt.xticks(rotation=90)
plt.show()

There are many columns/features with the label 'Unknown' which are useless, such as 'MSZoning_Unknown' , 'BsmtCond_Unknown' , 'BsmtExposure_Unknown', 'BsmtFinType1_Unknown', 'BsmtFinType2_Unknown', 'BsmtQual_Unknown' . This shows that filling null values ​​with 'Unknown' values ​​(in categorical data) is very bad.

in the future i want to improve it and do retraining with the best selected features

In [ ]:
# PREDICT MODEL AND SAVE SUBMISSION

y_pred = np.expm1(model.predict(x_test))

submission = pd.read_table(r'/kaggle/input/home-data-for-ml-course/sample_submission.csv', delimiter=',')   # LOAD SAMPLE SUBMISSION CSV

output = pd.DataFrame(y_pred, columns=['SalePrice'])
output = pd.concat([submission.iloc[:,0] , output], axis=1)
output.rename(columns={output.columns[0] : 'Id'}, inplace=True)

# SAVE SUBMISSION
output.to_csv('submission.csv', index=False)

<h4><strong style='color:skyblue;'> Thank you for reading this notebook till the end. Please upvote if you find it helpful. Thank You :)